# testing different cells dispositions in the space


In [1]:
# fileName

fileName = 'eg003_testing_dimensions'

In [2]:
# set libs

import pyNN.spiNNaker as sim
from pyNN import space 
import numpy as np
import matplotlib.pyplot as plt


Detected PyNN version 0.9.4 and Neo version 0.6.1


In [3]:
# simulation setting

dt         = 1          # (ms)
simtime    = 200       # (ms)
sim.setup(timestep=dt, 
          min_delay = 1,
          max_delay = 100) 


2021-04-10 15:25:06 INFO: Read cfg files: /home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spinn_front_end_common/interface/spinnaker.cfg, /home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spynnaker/pyNN/spynnaker.cfg, /home/spinnaker/.spynnaker.cfg
2021-04-10 15:25:06 INFO: Will search these locations for binaries: /home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spinn_front_end_common/common_model_binaries : /home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spynnaker/pyNN/model_binaries
2021-04-10 15:25:06 INFO: Setting time scale factor to 1.
2021-04-10 15:25:06 INFO: Setting machine time step to 1000 micro-seconds.


['/home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spinn_front_end_common/interface/spinnaker.cfg', '/home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spynnaker/pyNN/spynnaker.cfg', '/home/spinnaker/.spynnaker.cfg']


0

In [4]:
# network parameters

n_pop = 900     # Number of cells
weight = 0.05  # synaptic weight (µS)
syn_delay = 1

In [5]:
# making the network 

pops = {}

pops['1D'] = sim.Population(900, # number of cells
                       sim.IF_cond_exp, # cell model
                       sim.IF_cond_exp.default_parameters, # cell parameters
                       structure = space.Line(dx=1.0, 
                                              x0=0.0, 
                                              y=0.0, 
                                              z=0.0),
                       label="1D")


pops['2D'] = sim.Population(900, # number of cells
                       sim.IF_cond_exp, # cell model
                       sim.IF_cond_exp.default_parameters, # cell parameters
                       structure = space.Grid2D(
                                                aspect_ratio=1.0,
                                                dx=1.0,
                                                dy=1.0,
                                                x0=0.0,
                                                y0=0.0,
                                                z=0,
                                                fill_order='sequential',
                                                rng=None,
                                                ),
                      label="2D")

pops['3D'] = sim.Population(1000, # number of cells
                       sim.IF_cond_exp, # cell model
                       sim.IF_cond_exp.default_parameters, # cell parameters
                       structure = space.Grid3D(
                                                aspect_ratioXY=1.0,
                                                aspect_ratioXZ=1.0,
                                                dx=1.0,
                                                dy=1.0,
                                                dz=1.0,
                                                x0=0.0,
                                                y0=0.0,
                                                z0=0,
                                                fill_order='sequential',
                                                rng=None,
                                                ),
                       label="3D")



pops['Sphere'] = sim.Population(1000, # number of cells
                       sim.IF_cond_exp, # cell model
                       sim.IF_cond_exp.default_parameters, # cell parameters
                       structure = space.RandomStructure(boundary=space.Sphere(radius=100), 
                                                         origin=(0.0, 0.0, 0.0), 
                                                         rng=None),
                       label="Sphere")

pops['Cuboid'] = sim.Population(1000, # number of cells
                       sim.IF_cond_exp, # cell model
                       sim.IF_cond_exp.default_parameters, # cell parameters
                       structure = space.RandomStructure(boundary=space.Cuboid(width=10, 
                                                                               height=10, 
                                                                               depth=10), 
                                                         origin=(0.0, 0.0, 0.0), 
                                                         rng=None),
                       label="Cuboid")

In [17]:
# record state variables

for pop in pops.keys():
    pops[pop].record(['spikes','v','gsyn_exc','gsyn_inh'])   


In [6]:
# make projections

projs = {}

for pop in pops.keys():
    if pop == '1D':
        s = space.Space(axes = 'x')
    if pop == '2D':
        s = space.Space(axes = 'xy')
    if pop == '3D':
        s = space.Space(axes = 'xyz')
        
    projs[pop] = sim.Projection(
                                pops[pop],
                                pops[pop],
                                connector = sim.DistanceDependentProbabilityConnector('d<5'), 
                                synapse_type = sim.StaticSynapse(weight=weight, delay=syn_delay),
                                receptor_type = 'excitatory',
                                space = s
                            )


In [26]:
# make the thalamic input sources

popName = 'Thalamus'  

spike_times = [[]]*n_pop #list of spike lists, where one spike list is related to one spike source
random_sources_idx = [np.random.randint(n_pop*0.45, n_pop*0.55) for i in range(n_pop)]

for idx, sources in enumerate(random_sources_idx):
    spike_times[sources] = [abs(np.random.normal(loc=100, scale=10)) for n in range(10)]


pops[popName] = sim.Population(n_pop, 
                               sim.SpikeSourceArray(spike_times),
                               structure = space.Line(dx=1.0, 
                                              x0=0.0, 
                                              y=0.0, 
                                              z=0.0),
                               
                               label = popName)

pops[popName].record('spikes')

fig, axes = plt.subplots(1,1)
axes_list = fig.axes
axes_list[0].eventplot(spike_times)
axes_list[0].set_xlabel('[ms]')
axes_list[0].set_ylabel('thalamic spike sources')
axes_list[0].set_title('rasterplot')
axes_list[0].set_xlim(0, simtime)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(0.0, 200.0)

In [27]:
# make the thalamic projections

for pop in pops.keys():
    if pop != 'Thalamus':
            if pop == '1D':
                s = space.Space(axes = 'x')
            if pop == '2D':
                s = space.Space(axes = 'xy')
            if pop == '3D':
                s = space.Space(axes = 'xyz')
            print(pop)
            proj['Thalamus', pop] = sim.Projection(pops['Thalamus'], 
                                                     pops[pop],
                                                     connector = sim.DistanceDependentProbabilityConnector('d<5'),
                                                     synapse_type = sim.StaticSynapse(weight=0.08, delay=1),
                                                     receptor_type = 'excitatory',
                                                     space = s,
                                                     label = 'thalamus-exc connections')

1D
2D
3D
Sphere
Cuboid


In [28]:
# run simulation

sim.run(simtime) # simtime=500 [ms]

2021-04-09 17:59:29 INFO: Simulating for 200 1.0ms timesteps using a hardware timestep of 1000us
2021-04-09 17:59:29 INFO: Starting execution process
2021-04-09 17:59:33 INFO: Time 0:00:03.415725 taken by SpallocMaxMachineGenerator
Pre allocating resources for Extra Monitor support vertices
|0%                          50%                         100%|
2021-04-09 17:59:43 INFO: Time 0:00:10.127364 taken by PreAllocateResourcesForExtraMonitorSupport
Partitioning graph vertices
|0%                          50%                         100%|
Partitioning graph edges
|0%                          50%                         100%|
2021-04-09 17:59:48 INFO: Time 0:00:05.377534 taken by PartitionAndPlacePartitioner
Created spalloc job 5923965
2021-04-09 17:59:48 INFO: Created spalloc job 5923965
Waiting for board power commands to complete.
2021-04-09 17:59:48 INFO: Waiting for board power commands to complete.
2021-04-09 17:59:53 INFO: Time 0:00:05.051716 taken by SpallocAllocator
2021-04-09 1

200.0

In [29]:
for pop in pops.keys():
    if pop != 'Thalamus':
        print(len(proj['Thalamus', pop].get(['source', 'target', 'weight', 'delay'], format='list')))

2021-04-09 18:00:52 WARNING: Ignoring request to get source as with_address=True. 
2021-04-09 18:00:52 WARNING: Ignoring request to get target as with_address=True. 
Getting ['source', 'target', 'weight', 'delay']s for projection between Thalamus and 1D1
|0%                          50%                         100%|
2021-04-09 18:00:53 WARNING: Ignoring request to get source as with_address=True. 
2021-04-09 18:00:53 WARNING: Ignoring request to get target as with_address=True. 
Getting ['source', 'target', 'weight', 'delay']s for projection between Thalamus and 2D2
|0%                          50%                         100%|

8080


2021-04-09 18:00:53 WARNING: Ignoring request to get source as with_address=True. 
2021-04-09 18:00:53 WARNING: Ignoring request to get target as with_address=True. 
Getting ['source', 'target', 'weight', 'delay']s for projection between Thalamus and 3D3
|0%                          50%                         100%|

1131


2021-04-09 18:00:54 WARNING: Ignoring request to get source as with_address=True. 
2021-04-09 18:00:54 WARNING: Ignoring request to get target as with_address=True. 
Getting ['source', 'target', 'weight', 'delay']s for projection between Thalamus and Sphere4
|0%                          50%                         100%|
2021-04-09 18:00:54 WARNING: Ignoring request to get source as with_address=True. 
2021-04-09 18:00:54 WARNING: Ignoring request to get target as with_address=True. 
Getting ['source', 'target', 'weight', 'delay']s for projection between Thalamus and Cuboid5
|0%                          50%                         100%|
 

1433
9


2914


In [30]:
# save the results
outputs = {}

for pop in pops.keys():
    outputs[pop] = pops[pop].get_data()
    for recording in ['v', 'gsyn_inh', 'gsyn_exc', 'spikes']:
        pops[pop].write_data(fileName + '_' + str(pop) + '_' + str(recording) + '.pkl')

Getting spikes for 1D1
|0%                          50%                         100%|
Getting v for 1D1
|0%                          50%                         100%|
2021-04-09 18:00:55 WARNING: Warning getting data on a whole population when selective recording is active will result in only the requested neurons being returned in numerical order and without repeats.
Getting gsyn_exc for 1D1
|0%                          50%                         100%|
Getting gsyn_inh for 1D1
|0%                          50%                         100%|
Getting spikes for 1D1
|0%                          50%                         100%|
Getting v for 1D1
|0%                          50%                         100%|
Getting gsyn_exc for 1D1
|0%                          50%                         100%|
Getting gsyn_inh for 1D1
|0%                          50%                         100%|
Getting spikes for 1D1
|0%                          50%                         100%|
Getting v for 1D1
|0%   

In [31]:
# end simulation

sim.end()

2021-04-09 18:01:03 WARNING: During the extraction of data of 204800 bytes from memory address 1669697664, attempt 0 had 14 sequences that were lost.
2021-04-09 18:01:03 WARNING: During the extraction of data of 204800 bytes from memory address 1644867264, attempt 0 had 33 sequences that were lost.
2021-04-09 18:01:03 WARNING: During the extraction of data of 204800 bytes from memory address 1732368544, attempt 0 had 29 sequences that were lost.
2021-04-09 18:01:03 WARNING: During the extraction of data of 204800 bytes from memory address 1655462120, attempt 0 had 110 sequences that were lost.
2021-04-09 18:01:03 WARNING: During the extraction of data of 204800 bytes from memory address 1655462120, attempt 1 had 25 sequences that were lost.
2021-04-09 18:01:03 WARNING: During the extraction of data from chip (1, 1), there were 1 cases of serious data loss. The system recovered, but the speed of download was compromised. Reduce the number of executing applications and remove routers bet

In [32]:
def recover_results(outputs):
    results = {}
    for key in outputs.keys(): # to extract the name of the layer, e.g., Exc, Inh, Thalamus, etc  
        
        # to get voltage and conductances
        for analogsignal in outputs[key].segments[0].analogsignals:
            print(analogsignal.name)
            results[key, analogsignal.name] = analogsignal

        # to get spikes
        results[key, 'spikes'] = outputs[key].segments[0].spiketrains
    return results

In [33]:
# check results

results = recover_results(outputs)
results.keys()

v
gsyn_exc
gsyn_inh
v
gsyn_exc
gsyn_inh
v
gsyn_exc
gsyn_inh
v
gsyn_exc
gsyn_inh
v
gsyn_exc
gsyn_inh


dict_keys([('1D', 'v'), ('1D', 'gsyn_exc'), ('1D', 'gsyn_inh'), ('1D', 'spikes'), ('2D', 'v'), ('2D', 'gsyn_exc'), ('2D', 'gsyn_inh'), ('2D', 'spikes'), ('3D', 'v'), ('3D', 'gsyn_exc'), ('3D', 'gsyn_inh'), ('3D', 'spikes'), ('Sphere', 'v'), ('Sphere', 'gsyn_exc'), ('Sphere', 'gsyn_inh'), ('Sphere', 'spikes'), ('Cuboid', 'v'), ('Cuboid', 'gsyn_exc'), ('Cuboid', 'gsyn_inh'), ('Cuboid', 'spikes'), ('Thalamus', 'spikes')])

In [34]:
# check the spikes

fig, axes = plt.subplots(6, 1, figsize=(7,9), sharex=True)
fig.tight_layout(pad=5)
fig.suptitle('rasterplots')
axes_list = fig.axes

for idx, value in enumerate(['1D', '2D', '3D', 'Sphere', 'Cuboid', 'Thalamus']):
    axes_list[idx].eventplot(results[value, 'spikes'])
    axes_list[idx].set_title(str(value))
    axes_list[idx].set_xlabel('[ms]')
    axes_list[idx].set_ylabel('cells')
    axes_list[idx].set_xlim(0, simtime)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [35]:
# check the voltage

fig, axes = plt.subplots(1, 5, figsize=(9,5))
fig.tight_layout(pad=5)

axes_list = fig.axes

for idx, value in enumerate(['1D', '2D', '3D', 'Sphere', 'Cuboid']):
    im = axes_list[idx].imshow(results[value, 'v'].T)
    axes_list[idx].set_title('voltage of ' + str(value) + ' layer')
    axes_list[idx].set_xlabel('time [ms]')
    axes_list[idx].set_ylabel('cells')
    fig.colorbar(im, ax=axes_list[idx], fraction=0.010, label='[mV]')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [36]:
# check the conductances

for idx, value in enumerate(['1D', '2D', '3D', 'Sphere', 'Cuboid']):
    fig, axes = plt.subplots(1, 5)
    fig.tight_layout(pad=5)
    fig.suptitle(str(value) + ' layer')
    axes_list = fig.axes
    
    for idx, gsyn in enumerate(['gsyn_exc', 'gsyn_inh']):
        im = axes_list[idx].imshow(results[value, gsyn].T)
        axes_list[idx].set_title(str(gsyn))
        axes_list[idx].set_xlabel('time [ms]')
        axes_list[idx].set_ylabel('cells')
        fig.colorbar(im, ax=axes_list[idx], fraction=0.010, label='[uS]')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …